## A handy dandy trick for solvers

You have written quite a few algorithms to solve inverse problems. And I ask many questions regarding plot this and
plot that. How is convergence of this algorithm, vs that. And of course you can edit the solver to log you that information, but there are (from a software engineering point of view) nicer solutions.

Let's write a quick and dirty gradient descent algorithm to showcase that:

In [ ]:
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def gradient_descent(df, x0, step, iters):
    x = np.copy(x0)
    for _ in tqdm.trange(iters):
        x -= step * df(x)
    return x

The function I choose to optimize is the Rosenbrock function:

$$
f(x,y)=(x−1)^2+b (y−x1^2)^2
$$
and it's gradient:
$$
\nabla f = \begin{bmatrix} 
2(x - 1) - 4b(y−x^2) x \\
2b(y - x^2)
\end{bmatrix}
$$

In [ ]:
# for some b
b = 10;
f = lambda x,y: (x - 1) ** 2 + b * (y - x ** 2) ** 2;
df = lambda x,y: np.array([2 * (x - 1) - 4 * b * (y - x ** 2) * x, 2 * b * (y - x ** 2)])

In [ ]:
# Initialize figure 
fig = plt.figure(figsize=(12, 7))

# Depending on your matplotlib version:
ax = fig.add_subplot(projection='3d')
# or: 
# ax = fig.gca(projection='3d')

# Evaluate function
X = np.arange(-2, 2, 0.15)
Y = np.arange(-1, 3, 0.15)
X, Y = np.meshgrid(X, Y)
Z = f(X,Y)

# Plot the surface
surf = ax.plot_surface(X, Y, Z, cmap=cm.gist_heat_r, linewidth=0, antialiased=False)
ax.set_zlim(0, 200)
fig.colorbar(surf, shrink=0.5, aspect=10)
plt.show()

# Optimization:

In [ ]:
F = lambda X: f(X[0],X[1])
dF = lambda X: df(X[0],X[1])

In [ ]:
x0 = np.array([-1.4,1.1])
print(F(x0))
print(dF(x0))

# Initialize figure 
plt.figure(figsize=(12, 7))
plt.contour(X,Y,Z,200)
plt.plot([x0[0]],[x0[1]],marker='o',markersize=15, color ='r')

In [ ]:
xprime = gradient_descent(dF, x0, 0.01, 20)

# plot start and end point
plt.figure(figsize=(12, 7))
plt.contour(X,Y,Z,200)
plt.plot([x0[0]],[x0[1]],marker='o',markersize=15, color ='r')
plt.plot([xprime[0]],[xprime[1]],marker='o',markersize=15, color ='b')

Now, if your exercise is to show all the intermediate steps of the algorithm in the grap, you can modify the above gradient desent, to log that and return it as well. But then I ask for the convergence rate, and you need to adjust that as well, let us look at a more general approach: callbacks!

In [ ]:
def gradient_descent(df, x0, step, iters, callback=None):
    x = np.copy(x0)
    for i in tqdm.trange(iters):
        x -= step * df(x)
        
        if callback:
            callback(np.copy(x), i)
    return x

This is the same algorithm as above, but we take an optional argument, which you can use like this:

In [ ]:
xs = []
def callback1(x, i):
    xs.append(x)
    
xprime = gradient_descent(dF, x0, 0.01, 20, callback=callback1)

for x in xs:
    print(x)

In [ ]:
# How about a plot of the function values:
fs = []
def callback1(x, i):
    fs.append(F(x))
    
xprime = gradient_descent(dF, x0, 0.01, 20, callback=callback1)

plt.plot(fs)

And why pass the iteration number as well? If you run gradient descent for 1000 iterations for a tomographic reconstruction problem, you might not want to save each intermediate example, but only every kth one:

In [ ]:
# How about a plot of the function values:
fs = []
def callback1(x, i):
    if i % 10 == 0:
        fs.append(F(x))
    
xprime = gradient_descent(dF, x0, 0.005, 1000, callback=callback1)

plt.plot(fs)

The only thing missing is composition of callbacks. Currently, you can only use a single callback each call, but if you want to collect two differrent informations you are screwed. So, as an exercise create a callback class, that is composable, and overload e.g. the `__or__` operator (e.g. the pipe), to make it composable, such that a call like `cb = cb1 | cb2 | cb3` and you can pass it to the solver that way. The expected outcome of that call would be to call each of the three callbacks with the current guess and iteration number. 